# Establish baseline for media addtions

[T300706](https://phabricator.wikimedia.org/T300706)

Establish baselines for media addtions:
- how many images on average are added by each user in our target wikis (currently pt, ru)
- how many images on average are added by each user with >500 edits in our target wikis (currently pt, ru)

The method used to count media additions is from this ticket [T299712](https://phabricator.wikimedia.org/T299712) <br>
We include following media types: <br>
IMAGE_EXTENSIONS = ['.jpg', '.jpeg', '.png', '.svg', '.gif','.tif', '.bmp', '.webp', '.xcf','.djvu', '.pdf'] <br>
VIDEO_EXTENSIONS = ['.ogv', '.webm', '.mpg', '.mpeg'] <br>
AUDIO_EXTENSIONS = ['.ogg', '.mp3', '.mid', '.webm', '.flac', '.wav', '.oga'] <br>


In [2]:
import re

import wmfdata 
import math

import pandas as pd
import numpy as np

In [4]:
spark = wmfdata.spark.get_session(app_name='pyspark regular; media-changes',
                                  type='yarn-large', # local, yarn-regular, yarn-large
                                  )  

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


PYSPARK_PYTHON=/usr/lib/anaconda-wmf/bin/python3


SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/lib/spark2/jars/slf4j-log4j12-1.7.16.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/lib/hadoop/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/24 04:31:21 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
22/02/24 04:31:21 WARN Utils: Service 'sparkDriver' could not bind on port 12000. Attempting port 12001.
22/02/24 04:31:21 WARN Utils: Service 'sparkDriver' could not bind on port 12001. Attempting port 12002.
22/02/24 04:31:21 

In [5]:
print("Mediawiki partitions:")
spark.sql("show partitions wmf_raw.mediawiki_project_namespace_map").show(50, False)

Mediawiki partitions:
+------------------------+
|partition               |
+------------------------+
|snapshot=2016-12_private|
|snapshot=2017-07_private|
|snapshot=2021-08        |
|snapshot=2021-09        |
|snapshot=2021-10        |
|snapshot=2021-11        |
|snapshot=2021-12        |
|snapshot=2022-01        |
+------------------------+



## Parameters

In [6]:
mw_snapshot = '2021-12'  
wiki_dbs = ('ruwiki', 'ptwiki')
wiki_db_str = "('" + "','".join(wiki_dbs) + "')"  # otherwise single wiki leads to confusing syntax errors

start_timestamp = '2021-12-01'  # example format: 2008-04-04 15:04:29.0
end_timestamp = '2022-01-01'
media_list_table = 'cchen.media_nov_2021_l'


In [7]:
languages=['ruwiki', 'ptwiki'] #language editions to consider
#val=100 #threshold above which we consider images as non-icons


## Utils


In [8]:
MEDIA_PREFIXES = ['File', 'Image', 'Media']

MEDIA_ALIASES = {
    "ab": ["Медиа", "Файл", "Афаил", "Амедиа", "Изображение"],
    "ace": ["Beureukaih", "Gambar", "Alat", "Berkas"],
    "ady": ["Медиа"],
    "af": ["Lêer", "Beeld"],
    "als": ["Medium", "Datei", "Bild"],
    "am": ["ፋይል", "ስዕል"],
    "an": ["Imachen", "Imagen"],
    "ang": ["Ymele", "Biliþ"],
    "ar": ["ميديا", "صورة", "وسائط", "ملف"],
    "arc": ["ܠܦܦܐ", "ܡܝܕܝܐ"],
    "arz": ["ميديا", "صورة", "وسائط", "ملف"],
    "as": ["চিত্ৰ", "चित्र", "চিত্র", "মাধ্যম"],
    "ast": ["Imaxen", "Ficheru", "Imaxe", "Archivu", "Imagen", "Medios"],
    "atj": ["Tipatcimoctakewin", "Natisinahikaniwoc"],
    "av": ["Медиа", "Файл", "Изображение"],
    "ay": ["Medio", "Archivo", "Imagen"],
    "az": ["Mediya", "Şəkil", "Fayl"],
    "azb": ["رسانه", "تصویر", "مدیا", "فایل", "رسانه‌ای"],
    "ba": ["Медиа", "Рәсем", "Файл", "Изображение"],
    "bar": ["Medium", "Datei", "Bild"],
    "bat-smg": ["Vaizdas", "Medėjė", "Abruozdielis"],
    "bcl": ["Medio", "Ladawan"],
    "be": ["Мультымедыя", "Файл", "Выява"],
    "be-x-old": ["Мэдыя", "Файл", "Выява"],
    "bg": ["Медия", "Файл", "Картинка"],
    "bh": ["मीडिया", "चित्र"],
    "bjn": ["Barakas", "Gambar", "Berkas"],
    "bm": ["Média", "Fichier"],
    "bn": ["চিত্র", "মিডিয়া"],
    "bpy": ["ছবি", "মিডিয়া"],
    "br": ["Skeudenn", "Restr"],
    "bs": ["Mediji", "Slika", "Datoteka", "Medija"],
    "bug": ["Gambar", "Berkas"],
    "bxr": ["Файл", "Меди", "Изображение"],
    "ca": ["Fitxer", "Imatge"],
    "cbk-zam": ["Medio", "Archivo", "Imagen"],
    "cdo": ["文件", "媒體", "圖像", "檔案"],
    "ce": ["Хlум", "Медиа", "Сурт", "Файл", "Медйа", "Изображение"],
    "ceb": ["Payl", "Medya", "Imahen"],
    "ch": ["Litratu"],
    "ckb": ["میدیا", "پەڕگە"],
    "co": ["Immagine"],
    "crh": ["Медиа", "Resim", "Файл", "Fayl", "Ресим"],
    "cs": ["Soubor", "Média", "Obrázok"],
    "csb": ["Òbrôzk", "Grafika"],
    "cu": ["Видъ", "Ви́дъ", "Дѣло", "Срѣдьства"],
    "cv": ["Медиа", "Ӳкерчĕк", "Изображение"],
    "cy": ["Delwedd"],
    "da": ["Billede", "Fil"],
    "de": ["Medium", "Datei", "Bild"],
    "din": ["Ciɛl", "Apamduööt"],
    "diq": ["Medya", "Dosya"],
    "dsb": ["Wobraz", "Dataja", "Bild", "Medija"],
    "dty": ["चित्र", "मिडिया"],
    "dv": ["ފައިލު", "މީޑިއާ", "ފައިލް"],
    "el": ["Εικόνα", "Αρχείο", "Μέσο", "Μέσον"],
    "eml": ["Immagine"],
    "eo": ["Dosiero", "Aŭdvidaĵo"],
    "es": ["Medio", "Archivo", "Imagen"],
    "et": ["Pilt", "Fail", "Meedia"],
    "eu": ["Irudi", "Fitxategi"],
    "ext": ["Archivu", "Imagen", "Mediu"],
    "fa": ["رسانه", "تصویر", "مدیا", "پرونده", "رسانه‌ای"],
    "ff": ["Média", "Fichier"],
    "fi": ["Kuva", "Tiedosto"],
    "fiu-vro": ["Pilt", "Meediä"],
    "fo": ["Miðil", "Mynd"],
    "fr": ["Média", "Fichier"],
    "frp": ["Émâge", "Fichiér", "Mèdia"],
    "frr": ["Medium", "Datei", "Bild"],
    "fur": ["Immagine", "Figure"],
    "fy": ["Ofbyld"],
    "ga": ["Íomhá", "Meán"],
    "gag": ["Mediya", "Medya", "Resim", "Dosya", "Dosye"],
    "gan": ["媒体文件", "文件", "文檔", "档案", "媒體", "图像", "圖像", "媒体", "檔案"],
    "gd": ["Faidhle", "Meadhan"],
    "gl": ["Imaxe", "Ficheiro", "Arquivo", "Imagem"],
    "glk": ["رسانه", "تصویر", "پرونده", "فاىل", "رسانه‌ای", "مديا"],
    "gn": ["Medio", "Imagen", "Ta'ãnga"],
    "gom": ["माध्यम", "मिडिया", "फायल"],
    "gor": ["Gambar", "Berkas"],
    "got": ["𐍆𐌴𐌹𐌻𐌰"],
    "gu": ["દ્રશ્ય-શ્રાવ્ય (મિડિયા)", "દ્રશ્ય-શ્રાવ્ય_(મિડિયા)", "ચિત્ર"],
    "gv": ["Coadan", "Meanyn"],
    "hak": ["文件", "媒體", "圖像", "檔案"],
    "haw": ["Kiʻi", "Waihona", "Pāpaho"],
    "he": ["תמונה", "קו", "מדיה", "קובץ"],
    "hi": ["मीडिया", "चित्र"],
    "hif": ["file", "saadhan"],
    "hr": ["Mediji", "DT", "Slika", "F", "Datoteka"],
    "hsb": ["Wobraz", "Dataja", "Bild"],
    "ht": ["Imaj", "Fichye", "Medya"],
    "hu": ["Kép", "Fájl", "Média"],
    "hy": ["Պատկեր", "Մեդիա"],
    "ia": ["Imagine", "Multimedia"],
    "id": ["Gambar", "Berkas"],
    "ig": ["Nká", "Midia", "Usòrò", "Ákwúkwó orünotu", "Ákwúkwó_orünotu"],
    "ii": ["媒体文件", "文件", "档案", "图像", "媒体"],
    "ilo": ["Midia", "Papeles"],
    "inh": ["Медиа", "Файл", "Изображение"],
    "io": ["Imajo", "Arkivo"],
    "is": ["Miðill", "Mynd"],
    "it": ["Immagine"],
    "ja": ["メディア", "ファイル", "画像"],
    "jbo": ["velsku", "datnyvei"],
    "jv": ["Barkas", "Medhia", "Gambar", "Médhia"],
    "ka": ["მედია", "სურათი", "ფაილი"],
    "kaa": ["Swret", "Таспа", "سۋرەت", "Taspa", "Su'wret", "Сурет", "تاسپا"],
    "kab": ["Tugna"],
    "kbd": ["Медиа", "Файл"],
    "kbp": ["Média", "Fichier"],
    "kg": ["Fisye"],
    "kk": ["Swret", "سۋرەت", "Таспа", "Taspa", "Сурет", "تاسپا"],
    "kl": ["Billede", "Fiileq", "Fil"],
    "km": ["ឯកសារ", "រូបភាព", "មេឌា", "មីឌា"],
    "kn": ["ಚಿತ್ರ", "ಮೀಡಿಯ"],
    "ko": ["미디어", "파일", "그림"],
    "koi": ["Медиа", "Файл", "Изображение"],
    "krc": ["Медиа", "Файл", "Изображение"],
    "ks": ["میڈیا", "فَیِل"],
    "ksh": ["Beld", "Meedije", "Medie", "Belld", "Medium", "Datei", "Meedijum", "Bild"],
    "ku": ["میدیا", "پەڕگە", "Medya", "Wêne"],
    "kv": ["Медиа", "Файл", "Изображение"],
    "kw": ["Restren"],
    "ky": ["Медиа", "Файл"],
    "la": ["Imago", "Fasciculus"],
    "lad": ["Dossia", "Medya", "Archivo", "Dosya", "Imagen", "Meddia"],
    "lb": ["Fichier", "Bild"],
    "lbe": ["Медиа", "Сурат", "Изображение"],
    "lez": ["Медиа", "Mediya", "Файл", "Şəkil", "Изображение"],
    "lfn": ["Fix"],
    "li": ["Afbeelding", "Plaetje", "Aafbeilding"],
    "lij": ["Immaggine", "Immagine"],
    "lmo": ["Immagine", "Imàjine", "Archivi"],
    "ln": ["Média", "Fichier"],
    "lo": ["ສື່ອ", "ສື່", "ຮູບ"],
    "lrc": ["رسانه", "تصویر", "رسانه‌ای", "جانیا", "أسگ", "ڤارئسگأر"],
    "lt": ["Vaizdas", "Medija"],
    "ltg": ["Medeja", "Fails"],
    "lv": ["Attēls"],
    "mai": ["मेडिया", "फाइल"],
    "map-bms": ["Barkas", "Medhia", "Gambar", "Médhia"],
    "mdf": ["Медиа", "Няйф", "Изображение"],
    "mg": ["Rakitra", "Sary", "Média"],
    "mhr": ["Медиа", "Файл", "Изображение"],
    "min": ["Gambar", "Berkas"],
    "mk": ["Податотека", "Медија", "Медиум", "Слика"],
    "ml": ["പ്രമാണം", "ചി", "മീഡിയ", "പ്ര", "ചിത്രം"],
    "mn": ["Медиа", "Файл", "Зураг"],
    "mr": ["चित्र", "मिडिया"],
    "mrj": ["Медиа", "Файл", "Изображение"],
    "ms": ["Fail", "Imej"],
    "mt": ["Midja", "Medja", "Stampa"],
    "mwl": ["Multimédia", "Fexeiro", "Ficheiro", "Arquivo", "Imagem"],
    "my": ["ဖိုင်", "မီဒီယာ"],
    "myv": ["Медия", "Артовкс", "Изображение"],
    "mzn": ["رسانه", "تصویر", "مه‌دیا", "مدیا", "پرونده", "رسانه‌ای"],
    "nah": ["Mēdiatl", "Īxiptli", "Imagen"],
    "nap": ["Fiùra", "Immagine"],
    "nds": ["Datei", "Bild"],
    "nds-nl": ["Ofbeelding", "Afbeelding", "Bestaand"],
    "ne": ["मीडिया", "चित्र"],
    "new": ["किपा", "माध्यम"],
    "nl": ["Bestand", "Afbeelding"],
    "nn": ["Fil", "Bilde", "Filpeikar"],
    "no": ["Fil", "Medium", "Bilde"],
    "nov": [],
    "nrm": ["Média", "Fichier"],
    "nso": ["Seswantšho"],
    "nv": ["Eʼelyaaígíí"],
    "oc": ["Imatge", "Fichièr", "Mèdia"],
    "olo": ["Kuva", "Medii", "Failu"],
    "or": ["ମାଧ୍ୟମ", "ଫାଇଲ"],
    "os": ["Ныв", "Медиа", "Файл", "Изображение"],
    "pa": ["ਤਸਵੀਰ", "ਮੀਡੀਆ"],
    "pcd": ["Média", "Fichier"],
    "pdc": ["Medium", "Datei", "Bild", "Feil"],
    "pfl": ["Dadai", "Medium", "Datei", "Bild"],
    "pi": ["मीडिया", "पटिमा"],
    "pl": ["Plik", "Grafika"],
    "pms": ["Figura", "Immagine"],
    "pnb": ["میڈیا", "تصویر", "فائل"],
    "pnt": ["Εικόνα", "Αρχείον", "Εικόναν", "Μέσον"],
    "ps": ["انځور", "رسنۍ", "دوتنه"],
    "pt": ["Multimédia", "Ficheiro", "Arquivo", "Imagem"],
    "qu": ["Midya", "Imagen", "Rikcha"],
    "rm": ["Multimedia", "Datoteca"],
    "rmy": ["Fişier", "Mediya", "Chitro", "Imagine"],
    "ro": ["Fişier", "Imagine", "Fișier"],
    "roa-rup": ["Fişier", "Imagine", "Fișier"],
    "roa-tara": ["Immagine"],
    "ru": ["Медиа", "Файл", "Изображение"],
    "rue": ["Медіа", "Медиа", "Файл", "Изображение", "Зображення"],
    "rw": ["Dosiye", "Itangazamakuru"],
    "sa": ["चित्रम्", "माध्यमम्", "सञ्चिका", "माध्यम", "चित्रं"],
    "sah": ["Миэдьийэ", "Ойуу", "Билэ", "Изображение"],
    "sat": ["ᱨᱮᱫ", "ᱢᱤᱰᱤᱭᱟ"],
    "sc": ["Immàgini"],
    "scn": ["Immagine", "Mmàggini", "Mèdia"],
    "sd": ["عڪس", "ذريعات", "فائل"],
    "se": ["Fiila"],
    "sg": ["Média", "Fichier"],
    "sh": ["Mediji", "Slika", "Медија", "Datoteka", "Medija", "Слика"],
    "si": ["රූපය", "මාධ්‍යය", "ගොනුව"],
    "sk": ["Súbor", "Obrázok", "Médiá"],
    "sl": ["Slika", "Datoteka"],
    "sq": ["Figura", "Skeda"],
    "sr": ["Датотека", "Medij", "Slika", "Медија", "Datoteka", "Медиј", "Medija", "Слика"],
    "srn": ["Afbeelding", "Gefre"],
    "stq": ["Bielde", "Bild"],
    "su": ["Média", "Gambar"],
    "sv": ["Fil", "Bild"],
    "sw": ["Faili", "Picha"],
    "szl": ["Plik", "Grafika"],
    "ta": ["படிமம்", "ஊடகம்"],
    "tcy": ["ಮಾದ್ಯಮೊ", "ಫೈಲ್"],
    "te": ["ఫైలు", "దస్త్రం", "బొమ్మ", "మీడియా"],
    "tet": ["Imajen", "Arquivo", "Imagem"],
    "tg": ["Акс", "Медиа"],
    "th": ["ไฟล์", "สื่อ", "ภาพ"],
    "ti": ["ፋይል", "ሜድያ"],
    "tk": ["Faýl"],
    "tl": ["Midya", "Talaksan"],
    "tpi": ["Fail"],
    "tr": ["Medya", "Resim", "Dosya", "Ortam"],
    "tt": ["Медиа", "Рәсем", "Файл", "Räsem", "Изображение"],
    "ty": ["Média", "Fichier"],
    "tyv": ["Медиа", "Файл", "Изображение"],
    "udm": ["Медиа", "Файл", "Суред", "Изображение"],
    "ug": ["ۋاسىتە", "ھۆججەت"],
    "uk": ["Медіа", "Медиа", "Файл", "Изображение", "Зображення"],
    "ur": ["میڈیا", "تصویر", "وسیط", "زریعہ", "فائل", "ملف"],
    "uz": ["Mediya", "Tasvir", "Fayl"],
    "vec": ["Immagine", "Imàjine", "Mèdia"],
    "vep": ["Pilt", "Fail"],
    "vi": ["Phương_tiện", "Tập_tin", "Hình", "Tập tin", "Phương tiện"],
    "vls": ["Afbeelding", "Ofbeeldienge"],
    "vo": ["Ragiv", "Magod", "Nünamakanäd"],
    "wa": ["Imådje"],
    "war": ["Medya", "Fayl", "Paypay"],
    "wo": ["Xibaarukaay", "Dencukaay"],
    "wuu": ["文件", "档案", "图像", "媒体"],
    "xal": ["Аһар", "Боомг", "Изображение", "Зург"],
    "xmf": ["მედია", "სურათი", "ფაილი"],
    "yi": ["מעדיע", "תמונה", "טעקע", "בילד"],
    "yo": ["Fáìlì", "Amóhùnmáwòrán", "Àwòrán"],
    "za": ["媒体文件", "文件", "档案", "图像", "媒体"],
    "zea": ["Afbeelding", "Plaetje"],
    "zh": ["媒体文件", "F", "文件", "媒體", "档案", "图像", "圖像", "媒体", "檔案"],
    "zh-classical": ["文件", "媒體", "圖像", "檔案"],
    "zh-min-nan": ["tóng-àn", "文件", "媒體", "Mûi-thé", "圖像", "檔案"],
    "zh-yue": ["檔", "档", "文件", "图", "媒體", "圖", "档案", "图像", "圖像", "媒体", "檔案"],
}

# https://commons.wikimedia.org/wiki/Commons:File_types
IMAGE_EXTENSIONS = ['.jpg', '.jpeg', '.png', '.svg', '.gif','.tif', '.bmp', '.webp', '.xcf','.djvu', '.pdf']
VIDEO_EXTENSIONS = ['.ogv', '.webm', '.mpg', '.mpeg']
AUDIO_EXTENSIONS = ['.ogg', '.mp3', '.mid', '.webm', '.flac', '.wav', '.oga']
MEDIA_EXTENSIONS = list(set(IMAGE_EXTENSIONS + VIDEO_EXTENSIONS + AUDIO_EXTENSIONS))

In [9]:
exten_regex = ('(' + '|'.join([e + '\\b' for e in MEDIA_EXTENSIONS]) + ')').replace('.', '\.')
extension_pattern = re.compile(f'([\w ,\(\)\.-]+){exten_regex}', flags=re.UNICODE)
bracket_pattern = re.compile('(?<=\[\[)(.*?)(?=\]\])', flags=re.DOTALL)

# NOTE: I explored several approaches to this function and how they impacted speed:
# * mwparserfromhell parsing substantially increases processing time, even compared to many regexes
# * Reducing down the number of extensions considered has a very minimal impact on time
# * Removing the first regex that extracts links has a very minimal impact on time. In theory it should be mostly unnecessary but will catch some rare file extensions.
# * Ignoring upper-case file extensions (e.g., .JPG) by not lower-casing the wikitext and just doing .findall over the iterative .search has very little impact on time

def getMedia(wikitext, wiki_db='enwiki', max_link_length=240):
    """Gather counts of media files found directly in wikitext.
    
    See https://phabricator.wikimedia.org/T299712 for more details.
    Link length: https://commons.wikimedia.org/wiki/Commons:File_naming#Length
    """
    lang = wiki_db.replace('wiki', '')
    try:
        # find standard bracket-syntax links -- this likely could be dropped but adds minimal overhead
        med_prefixes = MEDIA_PREFIXES + MEDIA_ALIASES.get(lang, [])
        links = bracket_pattern.findall(wikitext)
        bracket_links = set([l.split(':', maxsplit=1)[1].split('|', maxsplit=1)[0].strip() for l in links if l.split(':', maxsplit=1)[0] in med_prefixes])
        
        # supplement with links outside brackets as determined via known file extensions
        # lower-case to handle e.g., .JPG instead of .jpg when searching for file extensions
        lc_wt = wikitext.lower()
        exten_links = []
        end = 0
        while True:
            m = extension_pattern.search(lc_wt, pos=end)
            if m is None:
                break
            start, end = m.span()
            exten_links.append(wikitext[start:end].strip())
        return [l.replace('\n', ' ') for l in bracket_links.union(exten_links) if len(l) <= max_link_length]
    except Exception:
        return None
    
spark.udf.register('getMedia', getMedia, 'ARRAY<String>')

<function __main__.getMedia(wikitext, wiki_db='enwiki', max_link_length=240)>

In [10]:
def compareMediaLists(curr_media, prev_media):
    """Compare two media lists to determine what changed."""
    try:
        changes = []
        unaligned = set(curr_media) ^ set(prev_media)
        for m in unaligned:
            if m in curr_media:
                changes.append((m, 1))
            elif m in prev_media:
                changes.append((m, -1))
        return changes
    except Exception:
        return None
    
spark.udf.register('compareMediaLists', compareMediaLists, 'ARRAY<STRUCT<filename:STRING, action:INT>>')

<function __main__.compareMediaLists(curr_media, prev_media)>

## Generate media lists

In [11]:
create_table_query = f"""
    CREATE TABLE IF NOT EXISTS {media_list_table} (
        wiki_db                         STRING         COMMENT 'Wiki -- e.g., enwiki for English',
        page_id                         INT            COMMENT 'Article page ID',
        user_text                       STRING         COMMENT 'User name of who made edit',
        revision_id                     BIGINT         COMMENT 'Revision ID',
        revision_parent_id              BIGINT         COMMENT 'Revision ID of parent revision',
        revision_is_identity_reverted   BOOLEAN        COMMENT 'Was revision reverted?',
        revision_is_identity_revert     BOOLEAN        COMMENT 'Did revision restore a previous revision?',
        revision_seconds_to_identity_revert    BIGINT        COMMENT 'seconds elapsed between revision posting and its revert',
        revision_tags                   ARRAY<STRING>  COMMENT 'Edit tags associated with revision',
        cur_rev_media_array             ARRAY<STRING>  COMMENT 'List of images in current revision',
        par_rev_media_array             ARRAY<STRING>  COMMENT 'List of images in parent revision'
    )
    """

spark.sql(create_table_query)
print(create_table_query)


    CREATE TABLE IF NOT EXISTS cchen.media_nov_2021_l (
        wiki_db                         STRING         COMMENT 'Wiki -- e.g., enwiki for English',
        page_id                         INT            COMMENT 'Article page ID',
        user_text                       STRING         COMMENT 'User name of who made edit',
        revision_id                     BIGINT         COMMENT 'Revision ID',
        revision_parent_id              BIGINT         COMMENT 'Revision ID of parent revision',
        revision_is_identity_reverted   BOOLEAN        COMMENT 'Was revision reverted?',
        revision_is_identity_revert     BOOLEAN        COMMENT 'Did revision restore a previous revision?',
        revision_seconds_to_identity_revert    BIGINT        COMMENT 'seconds elapsed between revision posting and its revert',
        revision_tags                   ARRAY<STRING>  COMMENT 'Edit tags associated with revision',
        cur_rev_media_array             ARRAY<STRING>  COMMENT 'List

In [23]:
"""
Explanation of CTEs:
* revisions: get all revisions + metadata from desired wikis / timeframe.
  * only main articles and filter out bots / anonymous users
* all_revision_ids: build deduplicated lists of all revision + parent revision IDs
* media_lists: for each revision ID, extract images from associated wikitext
* INSERT OVERWRITE...: join back in media lists with revisions + metadata

# TODO: are newlines in revision_text causing NULLs?
"""

print_for_hive = False
do_execute = True
wiki_db_str = "('" + "','".join(wiki_dbs) + "')"  # otherwise single wiki leads to confusing syntax errors

query = f"""
WITH revisions AS (
    SELECT
      wiki_db,
      page_id,
      event_user_text AS user_text,
      revision_id,
      revision_parent_id,
      revision_is_identity_reverted,
      revision_is_identity_revert,
      revision_seconds_to_identity_revert,
      revision_tags
    FROM wmf.mediawiki_history
    WHERE
      snapshot = '{mw_snapshot}'
      AND wiki_db IN {wiki_db_str}
      AND page_namespace = 0
      AND event_type = 'create'
      AND event_entity = 'revision'
      AND event_timestamp >= '{start_timestamp}'
      AND event_timestamp < '{end_timestamp}'
      AND SIZE(event_user_is_bot_by) < 1
      AND SIZE(event_user_is_bot_by_historical) < 1
      AND NOT event_user_is_anonymous
      AND NOT page_is_redirect
),
all_revision_ids AS (
    SELECT DISTINCT
      wiki_db,
      rev_id
    FROM (
        SELECT
          wiki_db,
          revision_id AS rev_id
        FROM revisions
        UNION ALL
        SELECT
          wiki_db,
          revision_parent_id AS rev_id
        FROM revisions
    ) r
),
media_lists AS (
    SELECT
      r.wiki_db,
      r.rev_id,
      getMedia(revision_text, wt.wiki_db) AS media_array
    FROM wmf.mediawiki_wikitext_history wt
    INNER JOIN all_revision_ids r
      ON (wt.wiki_db = r.wiki_db
          AND wt.revision_id = r.rev_id)
    WHERE
      snapshot = '{mw_snapshot}'
      AND wt.wiki_db IN {wiki_db_str}
)

INSERT OVERWRITE TABLE {media_list_table}     
SELECT
  r.wiki_db,
  page_id,
  user_text,
  revision_id,
  revision_parent_id,
  revision_is_identity_reverted,
  revision_is_identity_revert,
  revision_seconds_to_identity_revert,
  revision_tags,
  c.media_array AS cur_rev_media_array,
  p.media_array AS par_rev_media_array
FROM revisions r
LEFT JOIN media_lists c
  ON (r.wiki_db = c.wiki_db
      AND r.revision_id = c.rev_id)
LEFT JOIN media_lists p
  ON (r.wiki_db = p.wiki_db
      AND r.revision_parent_id = p.rev_id)
"""

if print_for_hive:
    print(re.sub(' +', ' ', re.sub('\n', ' ', query)).strip())
else:
    print(query)

if do_execute:
    result = spark.sql(query)


WITH revisions AS (
    SELECT
      wiki_db,
      page_id,
      event_user_text AS user_text,
      revision_id,
      revision_parent_id,
      revision_is_identity_reverted,
      revision_is_identity_revert,
      revision_seconds_to_identity_revert,
      revision_tags
    FROM wmf.mediawiki_history
    WHERE
      snapshot = '2021-11'
      AND wiki_db IN ('ruwiki','ptwiki')
      AND page_namespace = 0
      AND event_type = 'create'
      AND event_entity = 'revision'
      AND event_timestamp >= '2021-11-01'
      AND event_timestamp < '2021-12-01'
      AND SIZE(event_user_is_bot_by) < 1
      AND SIZE(event_user_is_bot_by_historical) < 1
      AND NOT event_user_is_anonymous
      AND NOT page_is_redirect
),
all_revision_ids AS (
    SELECT DISTINCT
      wiki_db,
      rev_id
    FROM (
        SELECT
          wiki_db,
          revision_id AS rev_id
        FROM revisions
        UNION ALL
        SELECT
          wiki_db,
          revision_parent_id AS rev_id
       

## Stats

In [12]:
##overall stats
#editors who made over 500 edits as ex_editors

print_for_hive = False
do_execute = True

query = f"""
    SELECT
      m.wiki_db,
      SUM(IF(revision_is_identity_reverted AND revision_seconds_to_identity_revert <= 172800 , 1, 0)) AS total_num_reverted,
      COUNT(1) AS total_num_edits,
      COUNT(DISTINCT(user_text)) AS total_num_editors,
      COUNT(DISTINCT(CASE WHEN user_editcount > 500 THEN user_text ELSE NULL END)) AS total_num_ex_editors 
    FROM {media_list_table} m
    LEFT JOIN wmf_raw.mediawiki_user u ON (m.user_text = u.user_name AND m.wiki_db = u.wiki_db)
    WHERE
      revision_id IS NOT NULL
    AND 
      u.snapshot = '{mw_snapshot}'
    AND 
      u.wiki_db in {wiki_db_str}
    GROUP BY
      m.wiki_db
"""

if print_for_hive:
    print(re.sub(' +', ' ', re.sub('\n', ' ', query)).strip())
else:
    print(query)

if do_execute:
    spark.sql(query).show(500, False)


    SELECT
      m.wiki_db,
      SUM(IF(revision_is_identity_reverted AND revision_seconds_to_identity_revert <= 172800 , 1, 0)) AS total_num_reverted,
      COUNT(1) AS total_num_edits,
      COUNT(DISTINCT(user_text)) AS total_num_editors,
      COUNT(DISTINCT(CASE WHEN user_editcount > 500 THEN user_text ELSE NULL END)) AS total_num_ex_editors 
    FROM cchen.media_nov_2021_l m
    LEFT JOIN wmf_raw.mediawiki_user u ON (m.user_text = u.user_name AND m.wiki_db = u.wiki_db)
    WHERE
      revision_id IS NOT NULL
    AND 
      u.snapshot = '2021-12'
    AND 
      u.wiki_db in ('ruwiki','ptwiki')
    GROUP BY
      m.wiki_db



+-------+------------------+---------------+-----------------+--------------------+
|wiki_db|total_num_reverted|total_num_edits|total_num_editors|total_num_ex_editors|
+-------+------------------+---------------+-----------------+--------------------+
|ruwiki |7363              |289511         |9881             |2591                |
|ptwiki |7184              |102345         |8097             |999                 |
+-------+------------------+---------------+-----------------+--------------------+



In [50]:
# Stats on subset of edits that are image changes
# editors who made over 500 edits as ex_editors

print_for_hive = False
do_execute = True

query = f"""
WITH changes AS (
    SELECT
      m.wiki_db,
      user_text,
      revision_id,
      IF(revision_is_identity_reverted AND revision_seconds_to_identity_revert <= 172800 , TRUE, FALSE) AS was_reverted,
      IF(user_editcount > 500, TRUE, FALSE) AS ex_editors, 
      INLINE(compareMediaLists(cur_rev_media_array, par_rev_media_array))
    FROM {media_list_table} m
    LEFT JOIN wmf_raw.mediawiki_user u ON (m.user_text = u.user_name AND m.wiki_db = u.wiki_db)
    WHERE
      revision_id IS NOT NULL
    AND 
      u.snapshot = '{mw_snapshot}'
    AND 
      u.wiki_db in {wiki_db_str}
),

change_counts AS (
    SELECT
      c.wiki_db,
      COUNT(DISTINCT(revision_id)) AS num_edits,
      COUNT(DISTINCT(IF(!was_reverted,revision_id, NULL))) AS num_non_reverted_edits,
      COUNT(DISTINCT(IF(!was_reverted AND ex_editors,revision_id, NULL))) AS num_non_reverted_edits_by_ex_editors,
      COUNT(DISTINCT(IF(!was_reverted, user_text, NULL))) AS num_editors,
      COUNT(DISTINCT(IF(ex_editors AND !was_reverted, user_text, NULL))) AS num_ex_editors, 
      SUM(IF(!was_reverted, 1, 0)) AS images_added,
      SUM(IF(!was_reverted AND ex_editors, 1, 0)) AS images_added_by_ex_editors
    FROM changes c
    WHERE action = 1
    GROUP BY
      c.wiki_db
)
SELECT
  c.wiki_db,
  num_edits,
  num_non_reverted_edits,  
  num_non_reverted_edits_by_ex_editors,
  num_editors,
  num_ex_editors,
  images_added,
  images_added_by_ex_editors
FROM change_counts c

"""

if print_for_hive:
    print(re.sub(' +', ' ', re.sub('\n', ' ', query)).strip())
else:
    print(query)

if do_execute:
    spark.sql(query).show(500, False)


WITH changes AS (
    SELECT
      m.wiki_db,
      user_text,
      revision_id,
      IF(revision_is_identity_reverted AND revision_seconds_to_identity_revert <= 172800 , TRUE, FALSE) AS was_reverted,
      IF(user_editcount > 500, TRUE, FALSE) AS ex_editors, 
      INLINE(compareMediaLists(cur_rev_media_array, par_rev_media_array))
    FROM cchen.media_nov_2021_l m
    LEFT JOIN wmf_raw.mediawiki_user u ON (m.user_text = u.user_name AND m.wiki_db = u.wiki_db)
    WHERE
      revision_id IS NOT NULL
    AND 
      u.snapshot = '2021-12'
    AND 
      u.wiki_db in ('ruwiki','ptwiki')
),

change_counts AS (
    SELECT
      c.wiki_db,
      COUNT(DISTINCT(revision_id)) AS num_edits,
      COUNT(DISTINCT(IF(!was_reverted,revision_id, NULL))) AS num_non_reverted_edits,
      COUNT(DISTINCT(IF(!was_reverted AND ex_editors,revision_id, NULL))) AS num_non_reverted_edits_by_ex_editors,
      COUNT(DISTINCT(IF(!was_reverted, user_text, NULL))) AS num_editors,
      COUNT(DISTINCT(IF(ex_edit

+-------+---------+----------------------+------------------------------------+-----------+--------------+------------+--------------------------+
|wiki_db|num_edits|num_non_reverted_edits|num_non_reverted_edits_by_ex_editors|num_editors|num_ex_editors|images_added|images_added_by_ex_editors|
+-------+---------+----------------------+------------------------------------+-----------+--------------+------------+--------------------------+
|ruwiki |13821    |13387                 |11048                               |2038       |1067          |24532       |20844                     |
|ptwiki |6815     |6458                  |4610                                |1204       |432           |10695       |8091                      |
+-------+---------+----------------------+------------------------------------+-----------+--------------+------------+--------------------------+



## Number of images added per user

In [27]:
query = f"""
SELECT
  m.wiki_db,
  user_text,
  revision_id,
  IF(revision_is_identity_reverted AND revision_seconds_to_identity_revert <= 172800 , TRUE, FALSE) AS was_reverted,
  IF(user_editcount > 500, TRUE, FALSE) AS ex_editors, 
  INLINE(compareMediaLists(cur_rev_media_array, par_rev_media_array))
FROM {media_list_table} m
LEFT JOIN wmf_raw.mediawiki_user u ON (m.user_text = u.user_name AND m.wiki_db = u.wiki_db)
WHERE
      revision_id IS NOT NULL
    AND 
      u.snapshot = '{mw_snapshot}'
    AND 
      u.wiki_db in {wiki_db_str} """

data = spark.sql(query).toPandas()

In [44]:
#median number of images added by each user

##ptwiki
data[(data["was_reverted"] == False) & (data["action"] == 1) & (data["wiki_db"] == "ptwiki")].groupby(['user_text']) .size() .describe()

count    1204.000000
mean        8.882890
std        28.382445
min         1.000000
25%         1.000000
50%         2.000000
75%         5.000000
max       453.000000
dtype: float64

In [45]:
##ruwiki
data[(data["was_reverted"] == False) & (data["action"] == 1) & (data["wiki_db"] == "ruwiki")].groupby(['user_text']) .size() .describe()

count    2038.000000
mean       12.037291
std        83.643659
min         1.000000
25%         1.000000
50%         2.000000
75%         7.000000
max      3358.000000
dtype: float64

In [47]:
#median number of images added by user with >500 edits

##ptwiki
data[(data["was_reverted"] == False) & (data["action"] == 1) & (data["ex_editors"] == True) & (data["wiki_db"] == "ptwiki")].groupby(['user_text']) .size().describe()
 

count    432.000000
mean      18.729167
std       44.187465
min        1.000000
25%        2.000000
50%        5.000000
75%       14.000000
max      453.000000
dtype: float64

In [48]:
##ruwiki
data[(data["was_reverted"] == False) & (data["action"] == 1) & (data["ex_editors"] == True) & (data["wiki_db"] == "ruwiki")].groupby(['user_text']) .size().describe()
 

count    1067.000000
mean       19.535145
std       114.793417
min         1.000000
25%         1.000000
50%         4.000000
75%        12.000000
max      3358.000000
dtype: float64